In [1]:
import numpy as np
import pandas as pd


In [2]:
from tqdm import notebook

def ts_oracle(point_coords, points_path):
    url = 'http://185.86.144.208/oracle?x1={:f}&x2={:f}&x3={:f}&x4={:f}&x5={:f}&x6={:f}&x7={:f}&x8={:f}&x9={:f}&x10={:f}'.format(*point_coords)
    
    with urlopen(url) as response, open(points_path, 'ab') as out_file:
        data = response.read().decode('utf-8')
        parsed_data = re.split(r'^Function value = |\nAttempts left = ', data)
        if (data != 'UNDIFINED'):
            result, attempts = [np.float(number)
                                for number in parsed_data if number]
        else:
            result = np.inf
        cur = np.append(point_coords, result)[np.newaxis, :]
        np.savetxt(out_file, cur)
    return result

In [3]:
oracle=ts_oracle

In [4]:
import tqdm
import os
import requests
import re
from urllib.request import urlopen
path = '.'
max_attempts = 5
comm_size = 10
start_size = 50
iterations = 300
sample_size = 1000000

eps = 0.1


x = np.zeros((start_size + iterations, 10))
x[:start_size] = np.random.uniform(low=0, high=10, size=(start_size, 10))
y = np.zeros(start_size + iterations)
    
assert(x.shape[0] == start_size + iterations)

for i in tqdm.tqdm(range(start_size)):
    y[i] = oracle(x[i], path)

for it in tqdm.tqdm(range(iterations)):
    p = np.random.rand(1)[0]
    if p < eps:
        for i in range(max_attempts):
            new_point = np.random.uniform(low=0, high=10, size=10)
            new_label = oracle(new_point, path)
            if new_label == np.inf:
                print('indef')
            else:
                break
    else:
        comm = [DecisionTreeRegressor() for i in range(comm_size)]
        kf = KFold(n_splits=comm_size, shuffle=True)
        k = 0
        cur_x = x[:start_size + it]
        cur_y = y[:start_size + it]
        for tr_idx, tt_idx in kf.split(cur_x):
            comm[k] = comm[k].fit(cur_x[tr_idx], cur_y[tr_idx])
            k += 1
        for i in range(max_attempts):
            X_rand = np.random.uniform(low=0, high=10, size=(sample_size, 10))
            pred = []
            for i in range(comm_size):
                pred.append(comm[i].predict(X_rand))
            new_point = X_rand[np.argmax(np.var(np.array(pred), axis=0))]
            new_label = oracle(new_point, path)
            
            
            if new_label == np.inf:
                print('undef')
            else:
                break
    
    x[it + start_size] = new_point
    y[it + start_size] = new_label 

In [5]:
path = 'mined_points'
df = pd.read_csv(path, header=None, sep=' ')
x, y = df.values[:, :-1], df.values[:, -1]

In [6]:
from sklearn.datasets import load_svmlight_file
test_path = 'points'
X_test, y_test = load_svmlight_file(test_path, multilabel=True)

In [7]:
X_test = X_test.toarray()

In [8]:
from sklearn.tree import DecisionTreeRegressor

In [9]:
model = DecisionTreeRegressor(random_state = 1337)
model.fit(x, y)
pred = model.predict(X_test)

count = pred.shape[0]
res = pd.DataFrame({'Id': [], 'Expected': []})
res['Id'] = np.arange(1, count + 1)
res['Expected'] = pred
res.to_csv('ans.csv', index=False)